In [53]:
import pandas as pd
import random

DATASET_URL = "https://raw.githubusercontent.com/ID2223KTH/id2223kth.github.io/master/assignments/lab1/titanic.csv"

data_df = pd.read_csv(DATASET_URL, index_col=None)
s0 = data_df.Survived[data_df['Survived'] == 0].sample(1).index
s1 = data_df.Survived[data_df['Survived'] == 1].sample(1).index

pick_random = random.uniform(0, 2)

# Pick a non-survivor
if pick_random < 1:
  titanic_df = data_df.iloc[s0[0]].to_frame().T.reset_index(drop=True)
  print("Non-survivor added")
# Pick a survivor
else:
  titanic_df = data_df.iloc[s1[0]].to_frame().T.reset_index(drop=True)
  print("Survivor added")

display(titanic_df)
type(titanic_df)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


Non-survivor added


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,649,0,3,"Willey, Mr. Edward",male,NaN,0,0,S.O./P.P. 751,7.55,NaN,S


pandas.core.frame.DataFrame

In [17]:
def generate_flower(name, sepal_len_max, sepal_len_min, sepal_width_max, sepal_width_min, 
                    petal_len_max, petal_len_min, petal_width_max, petal_width_min):
    """
    Returns a single iris flower as a single row in a DataFrame
    """
    import pandas as pd
    import random

    df = pd.DataFrame({ "sepal_length": [random.uniform(sepal_len_max, sepal_len_min)],
                       "sepal_width": [random.uniform(sepal_width_max, sepal_width_min)],
                       "petal_length": [random.uniform(petal_len_max, petal_len_min)],
                       "petal_width": [random.uniform(petal_width_max, petal_width_min)]
                      })
    df['variety'] = name
    return df


def get_random_iris_flower():
    """
    Returns a DataFrame containing one random iris flower
    """
    import pandas as pd
    import random

    virginica_df = generate_flower("Virginica", 8, 5.5, 3.8, 2.2, 7, 4.5, 2.5, 1.4)
    versicolor_df = generate_flower("Versicolor", 7.5, 4.5, 3.5, 2.1, 3.1, 5.5, 1.8, 1.0)
    setosa_df =  generate_flower("Setosa", 6, 4.5, 4.5, 2.3, 1.2, 2, 0.7, 0.3)

    # randomly pick one of these 3 and write it to the featurestore
    pick_random = random.uniform(0,3)
    if pick_random >= 2:
        iris_df = virginica_df
        print("Virginica added")
    elif pick_random >= 1:
        iris_df = versicolor_df
        print("Versicolor added")
    else:
        iris_df = setosa_df
        print("Setosa added")

    return iris_df

display(get_random_iris_flower())

Virginica added


,sepal_length,sepal_width,petal_length,petal_width,variety
0,5.565002,3.282845,4.520628,2.49895,Virginica


In [54]:
import numpy as np
data_df = pd.read_csv(DATASET_URL, index_col=0)

data_df = data_df.drop(['PassengerId'], axis=1)
data_df = data_df.drop(['Ticket'], axis=1)

# Cabin Feature
deck = {"A": 1, "B": 2, "C": 3, "D": 4, "E": 5, "F": 6, "G": 7, "U": 8}

data_df['Cabin'] = data_df['Cabin'].fillna("U0")
data_df['Deck'] = data_df['Cabin'].map(lambda x: re.compile("([a-zA-Z]+)").search(x).group())
data_df['Deck'] = data_df['Deck'].map(deck)
data_df['Deck'] = data_df['Deck'].fillna(0)
data_df['Deck'] = data_df['Deck'].astype(int)

# we can now drop the cabin feature
data_df = data_df.drop(['Cabin'], axis=1)

# Age Feature
mean = data_df["Age"].mean()
std = data_df["Age"].std()

is_null = data_df["Age"].isnull().sum()
# compute random numbers between the mean, std and is_null
rand_age = np.random.randint(mean - std, mean + std, size=is_null)
# fill NaN values in Age column with random values generated
age_slice = data_df["Age"].copy()
age_slice[np.isnan(age_slice)] = rand_age
data_df["Age"] = age_slice
data_df["Age"] = data_df["Age"].astype(int)

# Embarked Feature
most_common_val = data_df['Embarked'].describe()[2]
data_df['Embarked'] = data_df['Embarked'].fillna(most_common_val)

# Transforming / Converting Features
data_df['Fare'] = data_df['Fare'].astype(int)
genders = {"male": 0, "female": 1}
data_df['Sex'] = data_df['Sex'].map(genders)
ports = {"S": 0, "C": 1, "Q": 2}
data_df['Embarked'] = data_df['Embarked'].map(ports)

# Extract `Titles` from `Name` Feature
titles = {"Mr": 1, "Miss": 2, "Mrs": 3, "Master": 4, "Rare": 5}

# extract titles
data_df['Title'] = data_df.Name.str.extract(' ([A-Za-z]+)\.', expand=False)
# replace titles with a more common title or as Rare
data_df['Title'] = data_df['Title'].replace(['Lady', 'Countess', 'Capt', 'Col', 'Don', 'Dr',
                                              'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')
data_df['Title'] = data_df['Title'].replace('Mlle', 'Miss')
data_df['Title'] = data_df['Title'].replace('Ms', 'Miss')
data_df['Title'] = data_df['Title'].replace('Mme', 'Mrs')
# convert titles into numbers
data_df['Title'] = data_df['Title'].map(titles)
# filling NaN with 0, to get safe
data_df['Title'] = data_df['Title'].fillna(0)
data_df = data_df.drop(['Name'], axis=1)

data_df['Age_Class'] = data_df['Age'] * data_df['Pclass']

data_df['Relatives'] = data_df['SibSp'] + data_df['Parch']
data_df.loc[data_df['Relatives'] > 0, 'Not_alone'] = 0
data_df.loc[data_df['Relatives'] == 0, 'Not_alone'] = 1
data_df['Not_alone'] = data_df['Not_alone'].astype(int)

data_df['Fare_Per_Person'] = data_df['Fare'] / (data_df['Relatives'] + 1)
data_df['Fare_Per_Person'] = data_df['Fare_Per_Person'].astype(int)

# Add id for the feature store
data_df['Id'] = data_df.index

data_df['Age'] = data_df['Age'].astype("int64")
data_df['Age_Class'] = data_df['Age_Class'].astype("int64")
data_df['Deck'] = data_df['Deck'].astype("int64")
data_df['Embarked'] = data_df['Embarked'].astype("int64")
data_df['Fare'] = data_df['Fare'].astype("int64")
data_df['Fare_Per_Person'] = data_df['Fare_Per_Person'].astype("int64")
data_df['Id'] = data_df['Id'].astype("int64")
data_df['Not_alone'] = data_df['Not_alone'].astype("int64")
data_df['Parch'] = data_df['Parch'].astype("int64")
data_df['Pclass'] = data_df['Pclass'].astype("int64")
data_df['Relatives'] = data_df['Relatives'].astype("int64")
data_df['Sex'] = data_df['Sex'].astype("int64")
data_df['SibSp'] = data_df['SibSp'].astype("int64")
data_df['Survived'] = data_df['Survived'].astype("int64")
data_df['Title'] = data_df['Title'].astype("int64")

data_df

KeyError: "['PassengerId'] not found in axis"